This chapter deals with the application of deep learning to deal with *graph-structured* data.

# 13.1 Machine Learning on Graphs

We can broadly group applications of ML on graph-structured data based on its goal: to predict properties of nodes, of edges, or of the graph as a whole. 
- Node prediction: Classify documents according to their topics based on their hyperlinks and citations
- Edge prediction: Predict the presence of additional interactions (i.e. edges) within a protein network given prior knowledge about known interactions. 
- Whole graph prediction: Determine whether a molecule is water soluble. Here the target molecule is a graph, and we use a data set comprised of other graphs (molecules)

We can also learn useful internal representations of graphs, instead of doing plain prediction, in ***Graph Representation Learning***

GNNs define embedding vectors for each node in a graph which are then transformed through a series of learnable layers to create learned representations. We may also use learnable embeddings to represent the edges and even the graph as a whole. This is conceptually analogous to token and position embeddings in NLP DL applications.

## 13.1.1 Graph Properties

The text only deals with *simple graphs* possessing:
- At most one edge between each pair of nodes
- Only undirected edges
- No self-edges which connect nodes back to themselves

**Notation:**\
A graph $\mathcal G = (\mathcal V, \mathcal E)$ consists of a set of nodes (aka. vertices) $\mathcal V$ and edges $\mathcal E$.\
We index nodes by $n=1,...,N$ and denote an edge from node $n$ to $m$ as a tuple $(n,m)$\
Two nodes that are linked by an edge are called *neighbors*, and the set of all neighbors for node $n$ is denoted by $\mathcal N(n)$

The data associated with the nodes is structured into $D$-dimensional column vectors $x_n$ associated with each node $n$. The full data matrix is then $X$ with dimensionality $N\times D$ comprised of row vectors $x_n^\intercal$

## 13.1.2 Adjacency Matrix

Denoted by $A$, this is used to specify the edges in a graph. Given $N$ nodes, $A$ is an $N\times N$ symmetric matrix with each element denoting the presence of an edge *from* the node in row $i$ *to* the node in column $j$. Therefore, undirected graphs have symmetric adjacency matrices, while directed graphs have asymmetric adjacency matrices.

In [3]:
# example
import numpy as np
nodes = ['a', 'b', 'c', 'd']
edges = [('a', 'b'), ('b', 'a'), ('c', 'b'), ('c', 'd')]
A = np.zeros([4, 4])
for e in edges:
    i = nodes.index(e[0])
    j = nodes.index(e[1])
    A[i, j] = 1
A

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 1.],
       [0., 0., 0., 0.]])

**NOTE:**\
The adjacency matrix depends on the ordering of the nodes. If we changed the ordering of the nodes above such that $d$ preceded $a$, the matrix would change. Thus, the adjacency matrix is **varient** under **permutations** of the nodes.

## 13.1.3 Permutation Equivariance

The *permutation matrix* $P$ specifies a particular permutation of node ordering and has the same $N\times N$ shape as the adjacency matrix.\
It contains a single $1$ in each row *and* column (like Sudoku). The $1$ in position $n,m$ indicates that node $n$ will be relabeled as node $m$ after the permutation. 

Thus, the original data matrix $X$, in which the row vectors are the data vectors at each node, can be rearranged in accord with the permutation in $P$ as:
$$\tilde X = PX$$

In [12]:
X = np.vstack([np.random.randint(0, 10, 4) for _ in nodes])
print(X)
P = np.eye(len(nodes), len(nodes))
swaps = [('a', 'd')]  # swap a for d - yielding d, b, c, a
for s in swaps:
    i = nodes.index(s[0])
    j = nodes.index(s[1])
    P[i, i], P[j, j] = 0, 0  # clear diagonals
    P[i, j], P[j, i] = 1, 1  # indicate swap
print(P)
X = P @ X
print(X)

[[5 8 5 0]
 [6 6 9 4]
 [3 8 0 8]
 [7 3 6 9]]
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
[[7. 3. 6. 9.]
 [6. 6. 9. 4.]
 [3. 8. 0. 8.]
 [5. 8. 5. 0.]]


Then the adjacency matrix can be permuted by: $$\tilde A = PAP^\intercal$$
This ensures that *both* the rows *and* the columns are permuted.

In [13]:
A = P @ A @ P.T
print(A)

[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 1. 0. 0.]
 [0. 1. 0. 0.]]


When predicting for the entire graph want model predictions to be ***Invariant*** to the node label orderings, such that: $$y(\tilde X, \tilde A) = y(X, A)$$
Where $y(\cdot, \cdot)$ is the network output.

When predicting for individual nodes, we want the model predictions to be ***Equivariant*** to the node label orderings, such that the same node is always associated with the same prediction regardless of node order.
$$y(\tilde X, \tilde A) = Py(X, A)$$
Where $y(\cdot, \cdot)$ is a **vector** of network outputs, with one element per node.

# 13.2 Neural Message-Passing

So, there are some important properties that we want GNNs to possess.\
If they have node-level outputs, then their entire structure must be equivariant to node ordering. That is, each layer should be equivariant under node ordering.\
If they have graph-level outputs, then they must also be invariant to node ordering. This may be practically achieved by introducing a final output layer that is node-order invariant rather than making all layers invariant (so long as they are still equivariant).

Furthermore, because graphs can vary widely in size and complexity, we need GNNs to accept variable length inputs and to be scalable to large input sizes.

## 13.2.1 Convolutional Filters

We can apply convolutions to graphs somewhat like how we apply them to images. In an image, each pixel may be thought of as a node and each surrounding pixel may be thought to share an edge with the pixel.

A convolutional filter of order $k$, here, performs the following computation upon a *single element* (e.g. pixel) in the preceding layer $l$:
$$z_i^{(l+1)} = f\bigg( \sum_j^{k^2} w_j z_j^{(l)} + b \bigg)$$
Where $f(\cdot)$ is a non-linear activation function and the sum is taken over all $k^2$ elements in the convolutional patch. For instance, a $3\times 3$ convolution would sum over up-to $9$ pixels - the primary pixel and all $8$ pixels that surround it.

Note that this standard convolutional filter is not invariant under permutations of its elements. It can be made *equivariant* through modification:
$$z_i^{(l+1)} = f\bigg(w_{\text{neigh}} \sum_{j\in\mathcal{N}(i)}z_j^{(l)} + w_{\text{self}}z_i^{(l)} + b \bigg)$$
Where $\mathcal N(i)$ is the set of nodes neighboring node $i$ and $w_{\text{neigh}}$ is a weight assumed to be common across the set of neighbors